# MODS Phenotypes: Step 3. Load Grady into DuckDB

## `imports`

In [1]:
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
from IPython.display import display as display
from tqdm.auto import tqdm

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 16}

matplotlib.rc('font', **font)

In [2]:
sys.path.insert(0, "/opt/scratchspace/KLAB_SAIL/MODSPhenotypes/mods/")
from src.config import *
from src.utils import *
site_name = 'grady'

In [9]:
data_path = Path(f"{project_path}/data/{run_id}/extraction/{site_name}/")

## `config`

In [3]:
num_cpus = project_config['parameters']['num_cpus']
num_gpus = project_config['parameters']['num_gpus']
sepsis3_window_days = project_config["parameters"]["sepsis3_window_days"]

In [4]:
patient_id = project_config[site_name]["keys"]["patient_key"]
service_id = project_config[site_name]["keys"]["service_key"]
record_dt = project_config[site_name]["keys"]["record_dt"]

In [5]:
hospital_intime = project_config[site_name]["datetimes"]["hospital_intime"]
hospital_outtime = project_config[site_name]["datetimes"]["hospital_outtime"]
hospital_encounter_current = project_config[site_name]["datetimes"]["hospital_encounter_current"]
hospital_encounter_total = project_config[site_name]["datetimes"]["hospital_encounter_total"]

In [6]:
sepsis3_onset = project_config[site_name]["datetimes"]["sepsis3_onset"]
sepsis3_encounter = project_config[site_name]["datetimes"]["sepsis3_encounter"]
sepsis3_current = project_config[site_name]["datetimes"]["sepsis3_current"]
sepsis3_window = project_config[site_name]["datetimes"]["sepsis3_window"]

## `duckdb()`

### init `sql`

In [7]:
import duckdb
import sqlalchemy
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [8]:
%%sql
duckdb:////opt/scratchspace/KLAB_SAIL/MODSPhenotypes/data/MODS.duckdb
PRAGMA memory_limit='700GB';
PRAGMA threads=180;
PRAGMA enable_profiling;
PRAGMA enable_optimizer;
PRAGMA temp_directory='/opt/localdata/duck.tmp';

""


### Import `.parquet` files to `duckdb` tables

#### Static

In [ ]:
static_parquet = str(data_path / 'static_df_YEARLY' / 'static_df_*.parquet')

In [ ]:
%%sql
CREATE TABLE '{site_name}_static' AS
SELECT * FROM read_parquet('{static_parquet}');

In [ ]:
%sql ALTER TABLE {site_name}_static RENAME COLUMN pt_id TO pat_id;

#### Dynamic

In [ ]:
dynamic_parquet = str(data_path / 'dynamic_df_YEARLY' / 'dynamic_df_*.parquet')

In [ ]:
%%sql
CREATE TABLE '{site_name}_dynamic' AS
SELECT * FROM read_parquet('{dynamic_parquet}');

### Tables

1. Patient had sepsis during encounter
2. Patient data after admission (AND) before discharge
3. Investigate 28 days post-sepsis for MODS

In [ ]:
%%time
%%sql
CREATE OR REPLACE TABLE '{site_name}_sepsis' AS
SELECT *,
    date_part('year', {hospital_intime}) AS admission_year,
    1+CAST(date_diff('hour', {hospital_intime}, {hospital_outtime}) AS INTEGER) AS {hospital_encounter_total},
    1+CAST(date_diff('hour', {hospital_intime}, {record_dt}) AS INTEGER) AS {hospital_encounter_current},
    1+CAST(date_diff('hour', {sepsis3_onset}, {record_dt}) AS INTEGER) AS hours_after_sepsis

FROM {site_name}_static
INNER JOIN {site_name}_dynamic
ON {site_name}_static.{service_id} = {site_name}_dynamic.{service_id}

WHERE {sepsis3_onset} IS NOT NULL
AND {hospital_encounter_current} >= 0
AND {hospital_encounter_current} <= {hospital_encounter_total}
ORDER BY {site_name}_dynamic.{patient_id} ASC,
         {site_name}_dynamic.{service_id} ASC,
         {site_name}_dynamic.{record_dt} ASC;

In [ ]:
%%time
%%sql
CREATE OR REPLACE TABLE {site_name}_sepsis_28days AS
SELECT * FROM {site_name}_sepsis
WHERE hours_after_sepsis >= 0
AND hours_after_sepsis <= 672;

In [ ]:
%%time
%%sql
CREATE OR REPLACE TABLE {site_name}_sepsis_1day AS
SELECT * FROM {site_name}_sepsis_28days
WHERE hours_after_sepsis >= 0
AND hours_after_sepsis <= 24;

In [ ]:
%%time
%%sql
CREATE OR REPLACE TABLE {site_name}_sepsis_admission AS
(
    WITH sepsis_ranked AS (
    SELECT *, ROW_NUMBER() OVER(PARTITION BY sepsis.{patient_id}, sepsis.{service_id}
                                 ORDER BY sepsis.{hospital_encounter_current} ASC) AS rank
      FROM {site_name}_sepsis sepsis
    )
    SELECT * EXCLUDE (rank) FROM sepsis_ranked WHERE rank = 1
);

## Export/Import

### Export DB

In [21]:
export_path = Path(f"{project_path}/data/{run_id}/duckdb/{site_name}/")
export_path.mkdir(parents=True, exist_ok=True)

In [ ]:
%%time
%%sql
EXPORT DATABASE '{export_path}' (FORMAT PARQUET, COMPRESSION ZSTD, ROW_GROUP_SIZE 100000);

### Import DB
**Note: Import is not needed when using a .duckdb file**